In [1]:
!pip install pyDOE
!pip install cma


  Created wheel for pyDOE: filename=pyDOE-0.3.8-cp36-none-any.whl size=18178 sha256=bc086a417f7471e8fadbc086be861fca6c6656e46acb105b8962b281721dd273
  Stored in directory: /root/.cache/pip/wheels/7c/c8/58/a6493bd415e8ba5735082b5e0c096d7c1f2933077a8ce34544
Successfully built pyDOE
     |████████████████████████████████| 245kB 5.0MB/s 


In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import PolynomialFeatures
import bbobbenchmarks as bn
import sklearn

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F7 '''
def F7(X):
    f = bn.F7()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data

''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))
      

''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 0.33 ,random_state=0 , l1_ratio= 0.985 , fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 4.96 , C = 1000.000000 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_7_1000Samples.csv"
train_y = pd.read_csv(path).iloc[:,-1]
test_y = pd.read_csv('test_7_200Samples.csv').iloc[:,-1]
path = "train_latent_50D.csv"
train = pd.read_csv(path, index_col = 0)
test = pd.read_csv('test_latent_50D.csv', index_col = 0)
train ['Y'] = train_y
test ['Y'] = test_y
del train_y
del test_y
true = np.array(test['Y'])

## Surrogate Models

In [0]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

## CMA-ES

In [6]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,26):
        Cols.append(Columns[j]+'_Z'+str(i))

opt = cma.CMAOptions()
opt.set ("seed" , 0)
opt.set ("maxfevals" , 25000)

n_obs , dim =  30, 25
G = DOE(n_obs, dim)
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([30,75])
for i in range(G[1].shape[0]):
    print ('Run : '+ str(i))
    min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 0.1278 , options=opt) [0]
    min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 0.1278 , options=opt) [0]
    min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 0.1278 , options=opt) [0]
    X_Values [i,:] = list(min_kri)+list(min_svr)+list(min_eln)

Run : 0
(6_w,13)-aCMA-ES (mu_w=4.0,w_1=38%) in dimension 25 (seed=894904, Mon Apr 13 13:03:34 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     13 8.659182270949599e+03 1.0e+00 1.22e-01  1e-01  1e-01 0:00.0
    2     26 8.646658600890816e+03 1.1e+00 1.22e-01  1e-01  1e-01 0:00.0
    3     39 8.565543408538369e+03 1.1e+00 1.24e-01  1e-01  1e-01 0:00.0


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [8723.75984396] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [8703.82137822] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


  100   1300 -5.652027127986048e+03 1.7e+00 5.47e-03  4e-03  5e-03 0:00.9
  200   2600 -5.660926008548695e+03 2.0e+00 1.08e-04  7e-05  9e-05 0:01.8
  300   3900 -5.660929617447786e+03 2.2e+00 3.06e-06  2e-06  2e-06 0:02.8
  400   5200 -5.660929618682354e+03 2.3e+00 4.36e-08  2e-08  3e-08 0:03.7
  500   6500 -5.660929618682700e+03 2.5e+00 1.89e-09  8e-10  1e-09 0:04.6
  505   6565 -5.660929618682700e+03 2.5e+00 1.83e-09  8e-10  1e-09 0:04.7
termination on tolfun=1e-11 (Mon Apr 13 13:03:40 2020)
final/bestever f-value = -5.660930e+03 -5.660930e+03
incumbent solution: [-0.10824531  0.0218181   0.23798786 -0.32829555 -0.1673149  -0.17437785
  0.09472679  0.11535658 ...]
std deviations: [9.41719601e-10 9.19382655e-10 7.81938911e-10 9.41634567e-10
 9.15408355e-10 9.51624560e-10 9.37110137e-10 1.12854181e-09 ...]
(6_w,13)-aCMA-ES (mu_w=4.0,w_1=38%) in dimension 25 (seed=838506, Mon Apr 13 13:03:40 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     13 8.503

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [8507.88763528] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [8531.6504423] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


  100   1300 1.455058291469742e+03 1.8e+00 5.74e-03  5e-03  6e-03 0:00.8
  200   2600 1.449131170965396e+03 1.8e+00 7.18e-05  5e-05  6e-05 0:01.6
  300   3900 1.449130146839509e+03 1.9e+00 1.30e-06  7e-07  9e-07 0:02.3
  400   5200 1.449130146672861e+03 2.1e+00 1.78e-08  9e-09  1e-08 0:03.1
  500   6500 1.449130146672826e+03 2.7e+00 4.24e-09  2e-09  3e-09 0:03.9
  600   7800 1.449130146672831e+03 3.4e+00 3.89e-09  2e-09  2e-09 0:04.6
  700   9100 1.449130146672824e+03 4.2e+00 3.07e-09  1e-09  2e-09 0:05.4
  800  10400 1.449130146672823e+03 4.6e+00 2.40e-09  9e-10  2e-09 0:06.2
  900  11700 1.449130146672823e+03 5.3e+00 2.29e-09  9e-10  1e-09 0:07.0
 1000  13000 1.449130146672823e+03 6.5e+00 8.68e-10  3e-10  5e-10 0:07.7
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  14300 1.449130146672822e+03 8.0e+00 1.25e-09  5e-10  8e-10 0:08.5
 1200  15600 1.449130146672820e+03 1.0e+01 6.92e-10  2e-10  4e-10 0:09.3
 1300  16900 1.449130146672824e+03 1.1e+01 1.09e-09 

...

  100   1300 -5.605142880372006e+03 1.8e+00 1.34e-02  1e-02  1e-02 0:00.9
  200   2600 -5.660920467450256e+03 1.9e+00 2.10e-04  1e-04  2e-04 0:01.8
  300   3900 -5.660929617462290e+03 2.2e+00 2.59e-06  1e-06  2e-06 0:02.7
  400   5200 -5.660929618682127e+03 2.2e+00 6.40e-08  3e-08  4e-08 0:03.7
  500   6500 -5.660929618682703e+03 2.5e+00 2.31e-09  1e-09  1e-09 0:04.6
  532   6916 -5.660929618682698e+03 2.8e+00 2.69e-09  1e-09  2e-09 0:04.8
termination on tolfun=1e-11 (Mon Apr 13 13:14:58 2020)
final/bestever f-value = -5.660930e+03 -5.660930e+03
incumbent solution: [-0.1082453   0.0218181   0.23798786 -0.32829555 -0.16731489 -0.17437784
  0.09472679  0.11535657 ...]
std deviations: [1.66333739e-09 1.33733000e-09 1.23425386e-09 1.44205692e-09
 1.38058427e-09 1.43158935e-09 1.31356575e-09 1.63120913e-09 ...]
(6_w,13)-aCMA-ES (mu_w=4.0,w_1=38%) in dimension 25 (seed=955398, Mon Apr 13 13:14:58 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     13 8.547

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [8548.65127417] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [8548.81505648] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


  100   1300 1.461050062910375e+03 1.7e+00 8.80e-03  7e-03  8e-03 0:00.8
  200   2600 1.449132144460174e+03 1.8e+00 1.21e-04  8e-05  1e-04 0:01.5
  300   3900 1.449130146882838e+03 1.9e+00 1.38e-06  8e-07  1e-06 0:02.3
  400   5200 1.449130146672850e+03 1.9e+00 1.61e-08  8e-09  1e-08 0:03.1
  500   6500 1.449130146672829e+03 2.7e+00 7.49e-09  3e-09  5e-09 0:03.8
  600   7800 1.449130146672823e+03 3.3e+00 5.34e-09  2e-09  3e-09 0:04.6
  700   9100 1.449130146672819e+03 4.0e+00 1.66e-09  7e-10  1e-09 0:05.3
  800  10400 1.449130146672820e+03 5.5e+00 9.75e-10  4e-10  6e-10 0:06.1
  900  11700 1.449130146672826e+03 7.0e+00 5.63e-10  2e-10  4e-10 0:06.8
 1000  13000 1.449130146672826e+03 8.2e+00 6.99e-10  2e-10  5e-10 0:07.6
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  14300 1.449130146672821e+03 8.9e+00 1.25e-09  4e-10  1e-09 0:08.3
 1200  15600 1.449130146672820e+03 1.2e+01 9.71e-10  3e-10  1e-09 0:09.1
 1300  16900 1.449130146672825e+03 1.4e+01 2.04e-09 

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [13382.61105264] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [12537.52250571] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


   65    845 -4.865887609916405e+09 2.0e+00 1.35e+02  1e+02  1e+02 0:00.7
termination on tolfacupx=1000.0 (Mon Apr 13 13:15:14 2020)
final/bestever f-value = -4.251738e+09 -4.865888e+09
incumbent solution: [ 525.35019715  170.82722375 -358.06075314  458.85549275  447.64349552
  215.68081775  124.20754501   17.78781217 ...]
std deviations: [127.19579155 122.90758714 134.56650938 126.02128547 138.77607678
 130.6030654  120.81321623 126.97458927 ...]


In [7]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('F7_X_Values.csv')
Krig_Fun = np.zeros(30)
SVM_Fun = np.zeros(30)
ELN_Fun = np.zeros(30)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F7(X_Values.iloc[i,:25])
    SVM_Fun [i] = F7(X_Values.iloc[i,25:50])
    ELN_Fun [i] = F7(X_Values.iloc[i,50:75])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([382.18596256]), count=array([30])) 1.7053025658242404e-13
SVM
ModeResult(mode=array([360.39905553]), count=array([30])) 5.684341886080802e-14
ELN
ModeResult(mode=array([13725219.43525875]), count=array([1])) 9546836.74290575
